In [ ]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from itertools import product
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_squared_error
from math import sqrt


# ARIMA
Lags -> In auto regressive there are some value which is calleed lags, suppose today our output is Y it is actually dependent on some data on previous day so those previous days are lags.

Seasonal data -> In time series data, seasonality is the presence of variations that occur at specific regular intervals less than a year, such as weekly, monthly, or quarterly. Seasonality may be caused by various factors, such as weather, vacation, and holidays and consists of periodic, repetitive, and generally regular and predictable patterns in the levels of a time series.

In order to check our data is stationary or not by seeing the mean and the variance weather it almost same we basically apply something called Dickey–Fuller test.

Dickey–Fuller -> In statistics, the Dickey–Fuller test tests the null hypothesis that a unit root is present in an autoregressive time series model. The alternative hypothesis is different depending on which version of the test is used, but is usually stationarity or trend-stationarity. The test is named after the statisticians David Dickey and Wayne Fuller, who developed it in 1979.

NullHypothesis -> In inferential statistics, the null hypothesis is that two possibilities are the same. The null hypothesis is that the observed difference is due to chance alone. Using statistical tests, it is possible to calculate the likelihood that the null hypothesis is true.

In [ ]:
df = pd.read_excel('Infosys_Stock_Price_Dataset.xlsx', usecols=['Date', 'High'])
# df['Date'] = pd.to_datetime(df['Date'])
# df.set_index('Date',inplace=True)
# df.head()

In [ ]:
df.plot(x = 'Date', y = 'High', figsize=(16,8), title='Infosys daily stock price', grid=True, ylabel='High price (INR)', colormap='plasma')

Here we use adfuller from stats model for the dickey-fuller test. It takes the entire dataset and return five different values those are 'ADF Test Statistic','p-value','#Lags Used','Number of Observations Used'. The most important thing we require is p-value. Dickey-fuller test it is kind of hypothesis testing where in the Null hypothesis says that the data is not a stationary where as my alternate hypothesis says that it is stationary so, based on this particular condition is my p-value is < 0.05 it basically says that Data has no unit root and is stationary that means if we are getting 0.05 we are rejecting Nullhypothesis and if we are rejecting the nullhypothesis by deafult the alternate hypothesis get selected that is called as it is stationary. 
If our data is non stationary then we have to make our data stationary for that we can do Differencing.

Differencing -> In this method we are shifting the required number position of the price data or required data filed that means that number of record will move down. Once our data is stationary that means we are rejecting Null hypothesis and accepting the alternate hypothesis.

In [ ]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(price):
    result=adfuller(price)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(f'{label}: {value:.4f}')
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
adfuller_test(df['High'])

In [ ]:
autocorrelation_plot(df['High'])
plt.show()

In [ ]:
df['stationary_high'] = df['High'] - df['High'].shift(1)
df.head()

In [ ]:
adfuller_test(df['stationary_high'].dropna())

In [ ]:
df['stationary_high'].plot(figsize=(16,8), title='Infosys daily stock price', grid=True, ylabel='High price (INR)', colormap='plasma')

In [ ]:
autocorrelation_plot(df['stationary_high'])
plt.show()

Autocorrelation and Partial Autocorrelation
Identification of an AR model is often best done with the PACF.

For an AR model, the theoretical PACF “shuts off” past the order of the model. The phrase “shuts off” means that in theory the partial autocorrelations are equal to 0 beyond that point. Put another way, the number of non-zero partial autocorrelations gives the order of the AR model. By the “order of the model” we mean the most extreme lag of x that is used as a predictor.
Identification of an MA model is often best done with the ACF rather than the PACF.

For an MA model, the theoretical PACF does not shut off, but instead tapers toward 0 in some manner. A clearer pattern for an MA model is in the ACF. The ACF will have non-zero autocorrelations only at lags involved in the model.
p -> AR model lags
d -> differencing
q -> MA lags

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df['stationary_high'].iloc[1:],lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df['stationary_high'].iloc[1:],lags=40,ax=ax2)

In [ ]:
def get_best_pdq(df_data):
    flt_best_aic = None
    tup_best_pdq = None
    p = d = q = range(0, 5)
    lst_pdq = list(product(p, d, q))
    for tup_pdq in tqdm_notebook(lst_pdq):
        try:
            model_arima = ARIMA(df_data, order = tup_pdq)
            model_arima_fit = model_arima.fit()
            if flt_best_aic is None:
                flt_best_aic = model_arima_fit.aic
            elif model_arima_fit.aic < flt_best_aic:
                flt_best_aic = model_arima_fit.aic
                tup_best_pdq = tup_pdq
        except:
            continue
    return tup_best_pdq

In [ ]:
get_best_pdq(df['stationary_high'])

In [ ]:
model=ARIMA(df['High'], order=(4, 0, 4), trend='t')
model_fit=model.fit()
model_fit.summary()

In [ ]:
future_date = pd.date_range(start = df.Date.iloc[-1], periods=10, freq='D')
future_date_df = pd.DataFrame(future_date[1:], columns = ['Date'])
df = pd.concat([df, future_date_df], ignore_index = True)

In [ ]:
test_df = df.loc[2263:2282, :]
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.set_index('Date', inplace = True)
prediction = model_fit.predict(start = 2263, end = 2282)
prediction.index = test_df.index
# prediction = model_fit.predict(start = df.index[-15], end = df.index[-1])

In [ ]:
plt.figure(figsize = (15, 7.5), dpi = 400)
plt.xlabel('Date')
plt.title('Infosys daily stock price')
plt.ylabel('High price (INR)')
test_df['High'].plot(color = 'Blue', label = 'Actual data')
prediction.plot(color = 'red', label = 'ARIMA predicted')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
mse = mean_squared_error(df.High.iloc[2263:2274], model_fit.predict(start = 2263, end = 2273))
rmse = sqrt(mse)
print(f'Mean squared error: {mse}')
print(f'Root mean squared error: {rmse}')

# SARIMAX


In [ ]:
dfs = pd.read_excel('Infosys_Stock_Price_Dataset.xlsx', usecols=['Date', 'High'])

In [ ]:
dfs.plot(x = 'Date', y = 'High', figsize=(16,8), title='Infosys daily stock price', grid=True, ylabel='High price (INR)', colormap='plasma')

In [ ]:
plot_pacf(dfs['High'])
plot_acf(dfs['High'])

In [ ]:
ad_fuller_result = adfuller(dfs['High'])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

Since the p-value is large, we cannot reject the null hypothesis and must assume that the time series is non-stationary.
Now, let’s take the log difference in an effort to make it stationary:

In [ ]:
dfs['High'] = np.log(dfs['High'])
dfs['High'] = dfs['High'].diff()
dfs = dfs.drop(dfs.index[0])

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(dfs['High'])
plt.title("Log Difference of Daily Infosys stock price")
plt.show()

Now the p-value is small enough for us to reject the null hypothesis, and we can consider that the time series is stationary.

In [ ]:
ad_fuller_result = adfuller(dfs['High'])
print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

In [ ]:
plot_pacf(dfs['High'])
plot_acf(dfs['High'])

In [ ]:
def optimize_SARIMA(parameters_list, d, D, s, exog):
    """
        Return dataframe with parameters, corresponding AIC and SSE
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
        exog - the exogenous variable
    """
    
    results = []
    
    for param in tqdm_notebook(parameters_list):
        try: 
            model = SARIMAX(exog, order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        results.append([param, aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q)x(P,Q)', 'AIC']
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    
    return result_df

In [ ]:
p = range(0, 4, 1)
d = 1
q = range(0, 4, 1)
P = range(0, 4, 1)
D = 1
Q = range(0, 4, 1)
s = 4
parameters = product(p, q, P, Q)
parameters_list = list(parameters)
print(len(parameters_list))

In [ ]:
result_df = optimize_SARIMA(parameters_list, 1, 1, 4, dfs['High'])
result_df

In statistics, AIC is used to compare different possible models and determine which one is the best fit for the data. AIC is calculated from: the number of independent variables used to build the model. the maximum likelihood estimate of the model (how well the model reproduces the data).

In [ ]:
minm = result_df.AIC.min()
minm

In [ ]:
idx = [indx for indx, itm enumerate(result_df.AIC) if itm == result_df.AIC.min()]
result_df.iloc[idx[0]]

In [ ]:
best_model = SARIMAX(dfs['High'], order=(3, 1, 3), seasonal_order=(1, 1, 3, 4)).fit(dis=-1)
best_model.summary()

In [ ]:
best_model.plot_diagnostics(figsize=(15,12))

In [ ]:
future_date = pd.date_range(start = dfs.Date.iloc[-1], periods=10, freq='D')
future_date_dfs = pd.DataFrame(future_date[1:], columns = ['Date'])
dfs = pd.concat([dfs, future_date_dfs], ignore_index = True)

In [ ]:
test_dfs = dfs.loc[2263:2282, :]
test_dfs['Date'] = pd.to_datetime(test_dfs['Date'])
test_dfs.set_index('Date', inplace = True)
prediction_s = best_model.predict(start = 2263, end = 2282)
prediction_s.index = test_dfs.index
# prediction = model_fit.predict(start = df.index[-15], end = df.index[-1])

In [ ]:
plt.figure(figsize = (15, 7.5), dpi = 400)
plt.xlabel('Date')
plt.title('Infosys daily stock price')
plt.ylabel('High price (INR)')
test_dfs['High'].plot(color = 'Blue', label = 'Actual data')
prediction_s.plot(color = 'red', label = 'ARIMA predicted')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
mse_s = mean_squared_error(dfs.High.iloc[2263:2274], best_model.predict(start = 2263, end = 2273))
rmse_s = sqrt(mse_s)
print(f'Mean squared error: {mse_s}')
print(f'Root mean squared error: {rmse_s}')